NAMES & IDs: 
Bryant Liu - 
Jingyao Chen - A92071265
Joseph D’Angelo
Tammy Lee - A13573330

OVERVIEW: 
Include 3-4 sentences summarizing your group’s project 

RESEARCH QUESTION: 
What is your research question? (1-2 sentences) 

HYPOTHESIS: 
What is your main hypothesis and predictions? Briefly explain why. (2-3 sentences) 

BACKGROUND & PRIOR WORK: Why is this question of interest to your group? What background information led you to your hypothesis. Why is this important? 
Find some relevant prior work, and reference those sources. Even if you think you have a totally novel question, find the most similar prior work that you can and discuss how it relates to your project. 
References can be research publications, but they need not be. Blogs, GitHub repositories, company websites, etc., are all viable references if they are relevant to your project. (2-3 paragraphs, including at least 2 references) 




DATASET(S): 
What data will you use to answer your question? Describe the dataset(s) in terms of number of observations, what kind of features it contains, etc. You must use at least one dataset containing at least approximately 1000 observations (if your data are smaller but you feel they are sufficient, email Prof. Ellis). You are welcome (and in fact recommended) to find multiple datasets! If you do so, describe each one, and briefly explain how you will combine them together. Include the source of the dataset in the description here. 

DATA CLEANING: 
What methods did you use to analyze your data? Briefly explain what steps you had to take before you were able to use the datasets you chose to answer your question of interest. 
- How 'clean' is the data? - What did you have to do to get the data into a usable format? - What pre-processing steps that were required for your methods (for example, checking data distributions 
and performing any transformations that may be required) 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df_income = pd.read_csv('irs-zipcode.csv')
# Change column name using second row
df_income.columns = df_income.iloc[2]
# Drop 0th, 1st, nan rows; select 'Number of returns' and 'Amount' of 'Total income' columns 
df_income = df_income.drop([0, 1])
df_income = df_income.set_index('ZIP\ncode [1]')
df_income = df_income.drop(['ZIP\ncode [1]'])
df_income_selected = df_income.iloc[:, [16, 17]]
df_income_selected = df_income_selected.dropna()
df_income_selected.columns = ['Number of returns', 'Amount']
# Drop zip code 00000 and 99999
df_income_selected = df_income_selected.drop(['00000', '99999'])
# Drop all rows but the first, which is total returns and amount
df_income_selected = df_income_selected.loc[~df_income_selected.index.duplicated(keep='first')]
df_income_selected = df_income_selected.reset_index()
df_income_selected = df_income_selected.drop([0])

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ZIP code [1],Number of returns,Amount
1,90001,"21,670","619,635"
2,90002,"19,890","563,172"
3,90003,"27,290","730,810"
4,90004,"27,810","1,995,586"
5,90005,"15,850","807,743"
6,90006,"23,010","685,595"
7,90007,"12,620","382,443"
8,90008,"15,040","791,390"
9,90010,"2,430","533,018"
10,90011,"38,350","1,026,864"


DATA ANALYSIS & RESULTS: This section should include markdown text and code walking us through the following: 
- EDA - What distributions do your variables take? 
- Are there any outliers? - Relationship between variables? - Analysis (Note that you will likely have to do some Googling for analytical approaches not discussed in 
class. This is expected for this project and an important skill for a data scientist to master.) 
- What approaches did you use? Why? - What were the results? - What were your interpretation of these findings. 


There must be at least three appropriate data visualizations​throughout these sections. Each visualization must included an interpretation of what is displayed *and* what should be learned from that visualization. Be sure that the appropriate type of visualization is generated given the data that you have, axes are all labeled, and the visualizations clearly communicate the point you’re trying to make. 

ETHICS & PRIVACY: 
Briefly acknowledge and address any potential issues of ethics and privacy for the proposed project. In particular: 
- Did you have permission to use this data, for this purpose? - Are there privacy concerns regarding your datasets that you need to deal with, and/or terms of use that 
you need to comply with? - Are there potential biases in your dataset(s), in terms of who it composes, and how it was collected, that 
may be problematic in terms of it allowing for equitable analysis? (For example, does your data exclude particular populations, or is it likely to reflect particular human biases in a way that could be a problem?) - Are there any other issues related to your topic area, data, and/or analyses that are potentially problematic 
in terms of data privacy and equitable impact? - How did you handle issues you identified? 
(1-2 paragraphs) 


CONCLUSION & DISCUSSION: Discuss your project. Summarize your data and question. Briefly describe your analysis. Summarize your results and conclusions. Be sure to mention any limitations of your project. Discuss the impact of this work on society. (2-3 paragraphs) 